In [2]:
import cv2
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import math

from utils.line import *

In [3]:
img_path = "/home/rp24/file/test_img/3_1.png"
lower_hsv = np.array([0, 0, 240])   # 直线色域 0 0 245
upper_hsv = np.array([180, 255, 255])


frame = cv2.imread(img_path)
height, width, _ = frame.shape

lines = get_lines(frame, lower_hsv, upper_hsv)
cal = get_center_and_angle_len(lines)

cal.shape


(114, 4)

In [ ]:
plt.imshow(frame)

# Add lines on the frame
num_start = 0
num_end = num_start + 44
for idx in range(lines[num_start:num_end, ...].shape[0]):
    x0, y0, x1, y1 = lines[idx][0]
    center, angle = calculate_line_center_and_angle(x0, y0, x1, y1)
    print(f"center: {center}, angle: {angle:.1f}, {x0}, {y0} {x1}, {y1}")
    plt.plot([x0, x1], [y0, y1], color='red', linewidth=2)

# Show the plot
plt.show()

# capture

In [ ]:
cap = cv.VideoCapture(0)

output_dir = "/home/rp24/file/debug_output"

idx = 0

# ref

In [27]:
from utils.line import *
img_path = "/home/rp24/file/test_img/aw4.jpg"

lower_hsv = np.array([0, 0, 40])   # 直线色域 0 0 245
upper_hsv = np.array([180, 255, 255])

frame = cv2.imread(img_path)
frame = cv2.resize(frame,(600, 400),)
height, width, _ = frame.shape

lines = get_lines(frame, lower_hsv, upper_hsv)
cal = get_center_and_angle_len(lines)

cal.shape

(570, 4)

In [3]:
tt = cluster(cal)

In [5]:
cal0 = cal[tt == 0]
cal1 = cal[tt == 1]
print(f"ca0: {cal0.shape}, ca1: {cal1.shape}")

ca0: (20, 4), ca1: (41, 4)


In [20]:
def get_key_lines(lines, std_threshold):
    cal = get_center_and_angle_len(lines)
    var = np.var(cal, axis=0)
    n_lines = 1 if max(var[0], var[1]) < std_threshold else 2
    clusters = cluster(cal, n_lines)
    if n_lines == 1:
        return n_lines, lines[0], cal[0]
    else:
        cal0 = cal[clusters == 0]
        cal1 = cal[clusters == 1]
        lines0 = lines[clusters == 0]
        lines1 = lines[clusters == 1]
        return n_lines, (lines0[0], lines1[0]), (cal0[0], cal1[0])

In [22]:
a,b,c = get_key_lines(lines, 10)
print(a)
print(b)
print(c)

2
(array([282,  40, 502, 188], dtype=int32), array([281,  16, 599,  61], dtype=int32))
(array([392.        , 114.        ,  33.92979742, 265.14901471]), array([440.        ,  38.5       ,   8.05441445, 321.16818024]))


In [ ]:
def get_focus_area(frame, lines):
    pass

In [ ]:
import matplotlib.pyplot as plt
plt.figure()

img_path = "/home/rp24/file/test_img/aw3.jpg"
img = cv2.imread(img_path)
for cx, cy, angle in ca0:
    # cv2.circle(img, center=(cx, cy), color=(0, 0, 255), radius=20, thickness=2)
    cv2.circle(img, (cx, cy), (0, 0, 255), 20, 2)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()


In [ ]:
import cv2
img_path = "/home/rp24/file/test_img/aw3.jpg"
import matplotlib.pyplot as plt

# Load the image
img_path = "/home/rp24/file/test_img/aw3.jpg"
img = cv2.imread(img_path)

# Convert the image to different color spaces
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Display the images using matplotlib
plt.figure(figsize=(12, 6))

for i in range(3):
    plt.subplot(int(f"33{i*3+1}"))
    plt.imshow(img_rgb[..., i])
    plt.title("RGB")

for i in range(3):
    plt.subplot(int(f"33{i*3+2}"))
    plt.imshow(img_hsv[..., i])
    plt.title("HSV")

plt.subplot(int(f"133"))
plt.imshow(img_gray, cmap='gray')
plt.title("Grayscale")

plt.tight_layout()
plt.show()


# gpio

In [2]:
import RPi.GPIO as GPIO
import serial
# from serial import serial
import time
import struct
import numpy as np

In [2]:
port = "/dev/ttyAMA0"
# port = "/dev/ttyS0"

# 配置串口
com = serial.Serial(port, 9600) #发送提示字符
com.stopbits = 1
com.bytesize = 8
com.parity = 'N'

In [ ]:
# com.write(b'underwater robot')

com.write(b'\xFE\x01\x04\x00\x00\x00\x00\x00\x05\xFF')

def float2byte(f):
    return [hex(i) for i in struct.pack('f', f)]

angle = float(12.3)
# Convert angle to 2 bytes
angle_bytes = angle.to_bytes(2, byteorder='big')
# Send angle bytes
# com.write(angle_bytes)

while True:
    count = com.inWaiting()
    if count != 0:
    # 读取内容并打印
        recv = com.read(count)
        print(recv)
        #发回数据
        com.write(recv)
        # 清空接收缓冲区
        com.flushInput()
        # 延时 100ms,接收间隔
        time.sleep(0.1)
